<a href="https://colab.research.google.com/github/saichandrapandraju/QGEN_TAPAS_GPT/blob/main/qgen_tapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

     |████████████████████████████████| 2.0MB 8.8MB/s 
     |████████████████████████████████| 3.2MB 46.9MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ca71d93d4301c0565bf0bb164bf649354e9f872b92fa83c59d578804db753e73
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 8.9MB/s 


In [ ]:
from transformers import EncoderDecoderModel, TapasTokenizer, TapasForQuestionAnswering, AdamW, GPT2Tokenizer, GPT2Model
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
import requests, zipfile, io, os, ast, collections
import torch
# import datasets

In [ ]:
%%time
model = EncoderDecoderModel.from_encoder_decoder_pretrained('google/tapas-base', 'gpt2')

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.bias', 'h.0.crossattention.masked_bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.c_proj.bias', 'h.0.ln_cross_attn.weight', 'h.0.ln_cross_attn.bias', 'h.1.crossattention.bias', 'h.1.crossattention.masked_bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.c_proj.bias', 'h.1.ln_cross_attn.weight', 'h.1.ln_cross_attn.bias', 'h.2.crossattention.bias', 'h.2.crossattention.masked_bias', 'h.2.crossattention.c_attn.weight', 'h.2.crossattention.c_attn.bias', 'h.2.crossattention.q_attn.weight', 'h.2.crossattention.q_attn.bias', 'h.2.crossattention.c_proj.weight'

CPU times: user 21.6 s, sys: 3.26 s, total: 24.8 s
Wall time: 31.9 s


In [ ]:
tapas_tok = TapasTokenizer.from_pretrained('google/tapas-base')
gpt_tok = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
len(gpt_tok)

50257

In [ ]:
special_tok_dict = {'bos_token':'<BOS>','eos_token':'<EOS>','pad_token':'<PAD>'}
num_add_toks = gpt_tok.add_special_tokens(special_tokens_dict=special_tok_dict)
num_add_toks

3

In [ ]:
model.decoder.resize_token_embeddings(len(gpt_tok))

Embedding(50260, 768)

In [ ]:
!wget https://storage.googleapis.com/sqa-msft/SQA%20Release%201.0.zip

--2021-03-22 11:46:40--  https://storage.googleapis.com/sqa-msft/SQA%20Release%201.0.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4796932 (4.6M) [application/x-zip-compressed]
Saving to: ‘SQA Release 1.0.zip’

SQA Release 1.0.zip 100%[===================>]   4.57M  --.-KB/s    in 0.02s   

2021-03-22 11:46:40 (230 MB/s) - ‘SQA Release 1.0.zip’ saved [4796932/4796932]



In [ ]:
%%capture
!unzip SQA\ Release\ 1.0.zip

In [ ]:
!mv SQA\ Release\ 1.0 SQA

In [ ]:
!rm -r SQA\ Release\ 1.0.zip

In [ ]:
data = pd.read_csv('./SQA/train.tsv',sep='\t')

In [ ]:
data.head(2)

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"['(0, 4)', '(1, 4)', '(2, 4)', '(3, 4)', '(4, ...","['Louisiana State University', 'Valley HS (Las..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"['(0, 1)']",['Ben McDonald']


In [ ]:
def _parse_answer_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    coords = ast.literal_eval(answer_coordinate_str)
    # parse each string as a tuple
    for row_index, column_index in sorted(
        ast.literal_eval(coord) for coord in coords):
      answer_coordinates.append((row_index, column_index))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  
  return answer_coordinates


def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    for value in ast.literal_eval(answer_text):
      answer.append(value)
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer

data['answer_coordinates'] = data['answer_coordinates'].apply(lambda coords_str: _parse_answer_coordinates(coords_str))
data['answer_text'] = data['answer_text'].apply(lambda txt: _parse_answer_text(txt))


In [ ]:
data.head(2)

,id,annotator,position,question,table_file,answer_coordinates,answer_text
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve..."
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald]


In [ ]:
def get_sequence_id(example_id, annotator):
  if "-" in str(annotator):
    raise ValueError('"-" not allowed in annotator.')
  return f"{example_id}-{annotator}"

data['sequence_id'] = data.apply(lambda x: get_sequence_id(x.id, x.annotator), axis=1)
data.head()

,id,annotator,position,question,table_file,answer_coordinates,answer_text,sequence_id
0,nt-639,0,0,where are the players from?,table_csv/203_149.csv,"[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4...","[Louisiana State University, Valley HS (Las Ve...",nt-639-0
1,nt-639,0,1,which player went to louisiana state university?,table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-0
2,nt-639,1,0,who are the players?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
3,nt-639,1,1,which ones are in the top 26 picks?,table_csv/203_149.csv,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[Ben McDonald, Tyler Houston, Roger Salkeld, J...",nt-639-1
4,nt-639,1,2,"and of those, who is from louisiana state univ...",table_csv/203_149.csv,"[(0, 1)]",[Ben McDonald],nt-639-1


In [ ]:
# let's group table-question pairs by sequence id, and remove some columns we don't need 
grouped = data.groupby(by='sequence_id').agg(lambda x: x.tolist())
grouped = grouped.drop(columns=['id', 'annotator', 'position'])
grouped['table_file'] = grouped['table_file'].apply(lambda x: x[0])
grouped.head(2)

,question,table_file,answer_coordinates,answer_text
sequence_id,,,,
ns-1006-0,"[what are the game titles?, which of the games...",table_csv/203_583.csv,"[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, ...","[[Buggy Grand Prix: Kattobi! Dai-Sakusen, Gunb..."
ns-1006-1,"[what are the psikyo titles?, which of these h...",table_csv/203_583.csv,"[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, ...","[[Buggy Grand Prix: Kattobi! Dai-Sakusen, Gunb..."


In [ ]:
table_csv_path = './SQA/table_csv'

In [ ]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tapas_tokenizer, gpt_tokenizer, encoder_max_length = 512, decoder_max_length = 64):
        self.df = df
        self.tapas_tokenizer = tapas_tokenizer
        self.gpt_tokenizer = gpt_tokenizer
        self.encoder_max_length = encoder_max_length
        self.decoder_max_length = decoder_max_length
        self.encoding = {}

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.read_csv(table_csv_path + item.table_file[9:]).astype(str) # TapasTokenizer expects the table data to be text only
        
        previous_item = self.df.iloc[idx-1]
        tapas_encoding = self.tapas_tokenizer(table=table, 
                                  queries=None,
                                  answer_coordinates=item.answer_coordinates, 
                                  answer_text=item.answer_text,
                                  padding="max_length",
                                  truncation=True,
                                  return_tensors="pt")
        self.encoding['input_ids'] = tapas_encoding['input_ids'].squeeze(0)
        self.encoding['attention_mask'] = tapas_encoding['attention_mask'].squeeze(0)
        self.encoding['token_type_ids'] = tapas_encoding['token_type_ids'].squeeze(0)

        gpt_encoding = self.gpt_tokenizer(item.question, return_tensors = 'pt')
        temp = {key:val.squeeze(0) for key, val in gpt_encoding.items()}
        tmp_dec_ips = [self.gpt_tokenizer.bos_token_id] + temp['input_ids'].tolist() + [self.gpt_tokenizer.eos_token_id]
        tmp_dec_mask = [1] + temp['attention_mask'].tolist() + [1]

        self.encoding['decoder_input_ids'] = torch.LongTensor(tmp_dec_ips + ([self.gpt_tokenizer.pad_token_id]*(self.decoder_max_length-len(tmp_dec_ips))))
        self.encoding['decoder_attention_mask'] = torch.LongTensor(tmp_dec_mask + ([0]*(self.decoder_max_length-len(tmp_dec_mask))))
        self.encoding['labels'] = torch.LongTensor(tmp_dec_ips + ([self.gpt_tokenizer.eos_token_id]*(self.decoder_max_length-len(tmp_dec_ips))))


        return self.encoding

    def __len__(self):
        return len(self.df)


In [ ]:
simple_data = data[data['position'] == 0]
simple_data.shape

(5041, 8)

In [ ]:
train_df = simple_data.head(4500)
val_df = simple_data.tail(541).reset_index(drop=True)

In [ ]:
train_dataset = TableDataset(df=train_df, tapas_tokenizer=tapas_tok, gpt_tokenizer=gpt_tok)
val_dataset = TableDataset(df=val_df, tapas_tokenizer=tapas_tok, gpt_tokenizer=gpt_tok)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4)

In [ ]:
model.config.decoder_start_token_id = gpt_tok.bos_token_id
model.config.eos_token_id = gpt_tok.eos_token_id
model.config.pad_token_id = gpt_tok.pad_token_id

In [ ]:
%%capture
model.to('cuda')

In [42]:
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"].to('cuda')
        attention_mask = batch["attention_mask"].to('cuda')
        token_type_ids = batch["token_type_ids"].to('cuda')
        decoder_input_ids = batch["decoder_input_ids"].to('cuda')
        decoder_attention_mask = batch["decoder_attention_mask"].to('cuda')
        labels = batch["labels"].to('cuda')
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                        decoder_input_ids = decoder_input_ids, decoder_attention_mask = decoder_attention_mask, labels=labels)
        loss = outputs.loss
        print(f"Epoch - Loss : {str(epoch+1)} :: {loss.item()}")
        # print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


Epoch - Loss : 1 :: 21.754087448120117


TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


Epoch - Loss : 1 :: 19.338735580444336


TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


...

Epoch - Loss : 3 :: 0.5789250731468201


TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


Epoch - Loss : 3 :: 0.14357416331768036


TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.
TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


Epoch - Loss : 3 :: 0.2443181574344635


In [67]:
%%capture
model.eval()

TEST - 1

In [47]:
test1 = train_dataset[333]
test1['input_ids']

TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


tensor([  101,   102,  2161,  2504,  2407,  2173,  2541,  1013,  6185,  1018,
        23842,  4343,  2526,  1013,  6021,  1017, 23409,  2497,  3983,  2494,
         1013,  5840,  1018, 23842,  6280,  2432,  1013,  5709,  1018, 23842,
         3416,  2384,  1013,  5757,  1018, 23842,  3822,  2294,  1013,  5718,
         1018, 23842,  5504,  2289,  1013,  5511,  1018, 23842,  6280,  2263,
         1013,  5641,  1018, 23842,  6400,  2268,  1013,  2184,  1018, 23842,
         5351,  2230,  1013,  2340,  1018, 23842,  3708,  2249,  1013,  2260,
         1019, 19723,  1012,  3653,  2546,  1012,  4985,  2262,  1013,  2410,
         1020, 20720, 19723,  1012,  6286,  2286,  1013,  2403,  1020, 20720,
        19723,  1012,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [48]:
test1['labels']

tensor([50257, 10919,   373,   262,  1074,   338,  5290,  1622,    30, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258])

In [49]:
gpt_tok.decode(test1['labels'], skip_special_tokens = True)

"what was the team's worst season?"

In [51]:
test_out1 = model.generate(input_ids = test1['input_ids'].reshape((1,512)).to('cuda'), attention_mask = test1['attention_mask'].reshape((1,512)).to('cuda'))

In [52]:
test_out1

tensor([[50257, 10919,   389,   262,  3891,   286,   262,  3891,    30, 50258]],
       device='cuda:0')

In [53]:
gpt_tok.decode(test_out1[0].to('cpu'), skip_special_tokens = True)

'what are the names of the names?'

TEST - 2

In [54]:
test2 = val_dataset[3]
test2['input_ids']

TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


tensor([  101,   102, 13433,  2015,  2053,  4062,  2136, 10876,  2051,  1013,
         3394,  8370,  2685,  1015,  2403,  2728,  2341, 27698,  2891, 19808,
        17080,  2136,  3915, 10550,  1015,  1024,  3429,  1024,  5718,  1012,
         6079,  2581,  2321,  2861,  1016,  2538,  7663,  2140,  5553,  2072,
         1052,  2243,  2615,  3868, 10550,  1009,  1020,  1012,  1015, 10819,
         2015,  1019,  2676,  1017,  1017,  2030, 20282, 14262,  9035, 29434,
         2063,  3868, 10550,  1009,  1020,  1012,  1023, 10819,  2015,  1018,
         2656,  1018,  1019,  2097,  2373,  2136,  2660, 10550,  1009,  1021,
         1012,  1019, 10819,  2015,  2260,  2603,  1019,  1015, 28328,  8945,
         3126, 21351,  2015,  1050,  1013,  1044,  1013,  1048,  3868, 10550,
         1009,  1022,  1012,  1014, 10819,  2015,  1016,  2538,  1020,  1016,
         5846, 10958,  8865,  1050,  1013,  1044,  1013,  1048,  3868, 10550,
         1009,  1023,  1012,  1020, 10819,  2015,  1020,  2539, 

In [55]:
test2['labels']

tensor([50257,  8727,   389,   477,   286,   262,  6643,    30, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258])

In [56]:
gpt_tok.decode(test2['labels'], skip_special_tokens = True)

'who are all of the drivers?'

In [58]:
test_out2 = model.generate(input_ids = test2['input_ids'].reshape((1,512)).to('cuda'), attention_mask = test2['attention_mask'].reshape((1,512)).to('cuda'))

In [59]:
test_out2

tensor([[50257, 10919,   389,   262,  3891,   286,   262,  3891,    30, 50258]],
       device='cuda:0')

In [60]:
gpt_tok.decode(test_out2[0].to('cpu'), skip_special_tokens = True)

'what are the names of the names?'

TEST - 3

In [68]:
test3 = val_dataset[400]
test3['input_ids']

TAPAS is a question answering model but you have not passed a query. Please be aware that the model will probably not behave correctly.


tensor([  101,   102,  2171,  5907,  2287,  2013,  6139,  3396,  2769,  1006,
        13751,  1007,  3570, 13012,  7377,  2931,  2654,  4203,  9018,  1010,
         6187,  3063,  1002,  2656,  1010,  2199,  3453,  2414,  3287,  4805,
        11047,  1012,  9079,  1010, 19193,  2149,  3023,  3457,  1006,  3394,
         1007,  1002,  1014,  2439,  4748,  4360,  2931,  2423,  5862,  1010,
        11333, 15812,  1002,  1014,  5504,  4110,  1006,  2011, 11184, 19439,
         1007,  6326,  3287,  3590,  5529, 19022,  4215,  1010,  6187,  3076,
         1002,  1014,  5351,  4110,  1006,  2011,  5035,  1007,  4080,  3287,
         2538,  2417, 15422,  2080,  3509,  1010,  6187,  3076,  1013,  2166,
        18405,  1002,  1014,  4833,  4110,  1006,  2011,  5035,  1007, 10555,
         2931,  2382, 23689, 11733,  3490,  1010,  7632,  3076,  1002,  1014,
         4343,  4110,  1006,  2011,  3946,  1007,  1048,  6038,  2850,  2931,
         5354,  3050,  3349,  1010,  6187,  2236, 13666,  1002, 

In [69]:
test3['labels']

tensor([50257, 10919,   389,   477,   286,   262,  3891,    30, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258])

In [70]:
gpt_tok.decode(test3['labels'], skip_special_tokens = True)

'what are all of the names?'

In [71]:
test_out3 = model.generate(input_ids = test3['input_ids'].reshape((1,512)).to('cuda'), attention_mask = test3['attention_mask'].reshape((1,512)).to('cuda'))

In [72]:
test_out3

tensor([[50257, 10919,   389,   262,  3891,   286,   262,  3891,    30, 50258]],
       device='cuda:0')

In [73]:
gpt_tok.decode(test_out3[0].to('cpu'), skip_special_tokens = True)

'what are the names of the names?'